In [1]:
import torch 
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch.nn.modules.loss import NLLLoss
from torch.utils.data import DataLoader

In [2]:
train_set = datasets.FashionMNIST(root='./data/FashionMNIST', train=True, download=True, transform=transforms.ToTensor())
test_set = datasets.FashionMNIST(root='./data/FashionMNIST', train=False, download=True, transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=64, shuffle=True)


100%|██████████| 26421880/26421880 [00:17<00:00, 1527186.08it/s]


Extracting ./data/FashionMNIST\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data/FashionMNIST\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 159620.61it/s]


Extracting ./data/FashionMNIST\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data/FashionMNIST\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:03<00:00, 1389482.05it/s]


Extracting ./data/FashionMNIST\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data/FashionMNIST\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<00:00, 508017.72it/s]

Extracting ./data/FashionMNIST\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST\FashionMNIST\raw



In [3]:
class ClothesClassifier(nn.Module):
  def __init__(self, input_size, num_classes):
    super(ClothesClassifier, self).__init__()
    self.fc1 = nn.Linear(input_size, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 64)
    self.fc4 = nn.Linear(64, num_classes)
    #self.Dropout = nn.Dropout(p=0.1)


  def forward(self, x):
    x = x.reshape(x.shape[0], -1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.log_softmax(self.fc4(x), dim=1)
    return x


In [4]:
#some std. hyperparameters/variables
device = 'cuda' if torch.cuda.is_available() else 'cpu' #we will use a GPU if it is available
input_size = 784 #28*28
num_classes = 10
learning_rate = .001
epochs = 4


classifier = ClothesClassifier(input_size, num_classes).to(device)
criterion = NLLLoss()
optimizer = optim.Adam(classifier.parameters(), lr=learning_rate)


In [5]:
for epoch in range(epochs):
  for batch, (images, targets) in enumerate(train_loader):
    images, targets = images.to(device), targets.to(device)
    scores = classifier(images)
    #scores = classifier.forward(images)
    loss = criterion(scores, targets)
    optimizer.zero_grad() 
    loss.backward() 
    optimizer.step()


In [6]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Train accuracy")
  else:
    print('test accuracy')
  num_correct = 0
  num_samples = 0
  model.eval()
  with torch.no_grad():
    for x, y in loader:
      x, y = x.to(device), y.to(device)
      scores = model.forward(x) # scores is 64*10 in dimension
      _, predictions = scores.max(1)  # the label corresponding to the maximum label
				      # is taken as the output
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)
    print(f'Accuracy {float(num_correct)/float(num_samples)*100:.2f}')
  model.train()


check_accuracy(train_loader, classifier)
check_accuracy(test_loader, classifier)


Train accuracy
Accuracy 89.26
test accuracy
Accuracy 86.95
